In [ ]:

#  Notebook to load the DeepCDR graph ..
#



#  Setup: Display options ..

In [ ]:

%xmode Minimal

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate


print("--")


#  Setup: Connectivity ..

In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)
print(my_client.server_version)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
# ##################################################################

In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name = l_database.name).graphs_in_database():
      l_handle = my_client.get_database(name = l_database.name).get_graph_by_id(id = l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ONE SPECIFIC GRAPH ONLY

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name = l_database.name).graphs_in_database():
      if ((l_database.name == DB_NAME) and (l_graph.name == GRAPH_NAME)):   
         l_handle = my_client.get_database(name = l_database.name).get_graph_by_id(id = l_graph.graph_id)
         l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name = l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name = DB_NAME)

print(my_database.database_id)


In [4]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name = DB_NAME).create_graph(name = GRAPH_NAME, num_partitions = NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 6jFcKkJp4UZAvBZHRHVfaq14BcLwtAiYC3GCDMS7bFqx, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [ ]:
# ##################################################################

In [5]:

#  Now that we share servers, get a better list of databases and graphs ..
#

for l_each_d in my_client.databases():
   print("Database Name: %-28s   Id: %s" % (l_each_d.name, l_each_d.database_id))
      #
   for l_each_g in my_client.get_database(name=l_each_d.name).graphs_in_database(): 
      print("   Graph Name: %-28s      Version: %-8d   Id: %s" % (l_each_g.name, l_each_g.version, l_each_g.graph_id))


print(""  )
print("--")


Database Name: my_db                          Id: 4MAP8VHNkHLJQX6VhgBzqgpubGj5ot8c84iWJxY2vG2p
   Graph Name: my_graph                          Version: 0          Id: 6jFcKkJp4UZAvBZHRHVfaq14BcLwtAiYC3GCDMS7bFqx
   Graph Name: my_graph_BB                       Version: 1          Id: EDJ39S7aoeT2aT2BchYPHzwHB14JUfCLdZ2HKAS6eDqn
Database Name: default                        Id: QnXF6oF6UFgJhbwqvu7VxmSprvKxMakG3wkMW5jMFqg

--


In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:

my_graph.query("CALL graph.schema() RETURN *")


In [ ]:

l_rows = 10

l_query  = """
   MATCH (n) 
   RETURN n
   LIMIT {0}
   """.format(l_rows)

l_result = my_graph.query(l_query)

print(l_result)


#  Step 01: Process CSV files ..

In [ ]:

#  Read the CSVs as Step 01 towards populating our graph
#

l_folder   = "./10_Data/"


#  Nodes
#
df_PatientVisitNode                       = pd.read_csv(l_folder + "N_01_00_PatientVisit.txt"                   , header = "infer", delimiter = "|")
df_MedicalSpecialtyNode                   = pd.read_csv(l_folder + "N_02_00_MedicalSpecialty.txt"               , header = "infer", delimiter = "|")
df_KeywordNode                            = pd.read_csv(l_folder + "N_03_00_Keyword.txt"                        , header = "infer", delimiter = "|")
   #
df_UmlsEntityNode                         = pd.read_csv(l_folder + "N_04_00_UmlsEntity.txt"                     , header = "infer", delimiter = "|")
df_UmlsVocabularyNode                     = pd.read_csv(l_folder + "N_05_00_UmlsVocabulary.txt"                 , header = "infer", delimiter = "|")

l_nodecnt = (
   len(df_PatientVisitNode    ) + 
   len(df_MedicalSpecialtyNode) + 
   len(df_KeywordNode         ) + 
   len(df_UmlsEntityNode      ) + 
   len(df_UmlsVocabularyNode  ) 
   )


#  Edges
#
df_PatientVisitToMedicalSpecialtyEdge_N   = pd.read_csv(l_folder + "E_01_00_N_PatientVistToMedicalSpecialty.txt", header = "infer", delimiter = "|")
df_PatientVisitToMedicalSpecialtyEdge_S   = pd.read_csv(l_folder + "E_01_00_S_PatientVistToMedicalSpecialty.txt", header = "infer", delimiter = "|")
   #
df_PatientVisitToKeywordEdge_N            = pd.read_csv(l_folder + "E_02_00_N_PatientVistToKeyword.txt"         , header = "infer", delimiter = "|")
df_PatientVisitToKeywordEdge_S            = pd.read_csv(l_folder + "E_02_00_S_PatientVistToKeyword.txt"         , header = "infer", delimiter = "|")

df_PatientVisitToEntityEdge_N             = pd.read_csv(l_folder + "E_03_00_N_PatientVistToEntity.txt"          , header = "infer", delimiter = "|")
df_PatientVisitToEntityEdge_S             = pd.read_csv(l_folder + "E_03_00_S_PatientVistToEntity.txt"          , header = "infer", delimiter = "|")
   #
df_EntityToVocabularyEdge_N               = pd.read_csv(l_folder + "E_04_00_N_PatientVistToVocabulary.txt"      , header = "infer", delimiter = "|")
df_EntityToVocabularyEdge_S               = pd.read_csv(l_folder + "E_04_00_S_PatientVistToVocabulary.txt"      , header = "infer", delimiter = "|")

l_edgecnt = (
   len(df_PatientVisitToMedicalSpecialtyEdge_N) +
   len(df_PatientVisitToMedicalSpecialtyEdge_S) +
   len(df_PatientVisitToKeywordEdge_N         ) +
   len(df_PatientVisitToKeywordEdge_S         ) +
   len(df_PatientVisitToEntityEdge_N          ) +
   len(df_PatientVisitToEntityEdge_S          ) +
   len(df_EntityToVocabularyEdge_N            ) +
   len(df_EntityToVocabularyEdge_S            )
)


print("--")





        
print("")
print("--")

